In [1]:
%load_ext autoreload
%autoreload 2

import os

os.environ["CYCLONEDDS_URI"] = "file:///home/FlexivPy/cyclonedds_v0.xml"

In [ ]:
from FlexivPy.joy import XBoxController

joy = XBoxController(0)
joy.getStates()

Before going through the next cells, start the asynchronous simulator or the real robot bridge by tunning the following in your terminal:

**Asynchronous Simulator:**

```bash
flexivpy_async_sim --mode velocity
```

**Real Robot Client**

```bash
CYCLONEDDS_URI=file:///home/FlexivPy/cyclonedds_v0.xml robot_server -cm 3  --path /home/FlexivPy/FlexivPy/assets/ -rcf /home/FlexivPy/flexivpy_bridge/config.yaml
```

The `cm` 3 means that the robot is started in joint velocity mode.

** Camera **
```bash
CYCLONEDDS_URI=file:///home/FlexivPy/cyclonedds_v0.xml python FlexivPy/async_camera_app.py

CYCLONEDDS_URI=file:///home/FlexivPy/cyclonedds_v0.xml python FlexivPy/async_camera_app_multiple.py --camera_id 0 1 2

```

In [ ]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
import time
from FlexivPy.robot.dds.subscriber import get_last_msg
import cv2
from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from cyclonedds.sub import Subscriber, DataReader
import time

from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from FlexivPy.robot.dds.flexiv_messages import EnvImage
from datetime import datetime
import matplotlib.pyplot as plt
from FlexivPy.camera_dds_client import CameraDDSClient
from functools import partial


model = FlexivModel()
robot = FlexivDDSClient()
# Run this in another terminal!
# python FlexivPy/async_camera_app.py

camera1 = CameraDDSClient(camera_topic="EnvImage_0")
camera2 = CameraDDSClient(camera_topic="EnvImage_1")
camera3 = CameraDDSClient(camera_topic="EnvImage_2")


def cut_image(image, start_x, start_y, size):
    end_y = start_y + size
    end_x = start_x + size

    interest_area = image[start_y:end_y, start_x:end_x]
    if interest_area.shape[0] != size or interest_area.shape[1] != size:
        raise ValueError(
            "Interest area is not the correct size. Expected size: {}, Actual size: {}".format(
                size, interest_area.shape
            )
        )
    interest_area_6464 = cv2.resize(
        interest_area, (64, 64), interpolation=cv2.INTER_AREA
    )
    return interest_area_6464, interest_area

In [ ]:
image1 = camera1.get_env_image()
image2 = camera2.get_env_image()
image3 = camera3.get_env_image()
plt.rcParams["figure.dpi"] = 400


cut_image1 = partial(cut_image, start_x=75, start_y=90, size=135)
cut_image2 = partial(cut_image, start_x=40, start_y=10, size=230)
cut_image3 = partial(cut_image, start_x=50, start_y=20, size=220)

fig, ax = plt.subplots(3, 3)
ax[0, 0].imshow(image1)
ax[0, 1].imshow(image2)
ax[0, 2].imshow(image3)

# rigth camera.
r_64, r_interest = cut_image1(image1)
c_64, c_interest = cut_image2(image2)
l_64, l_interest = cut_image3(image3)

ax[1, 0].imshow(r_interest)
ax[1, 1].imshow(c_interest)
ax[1, 2].imshow(l_interest)

# store the interest areas
cv2.imwrite("r_interest.png", cv2.cvtColor(r_interest, cv2.COLOR_RGB2BGR))
cv2.imwrite("c_interest.png", cv2.cvtColor(c_interest, cv2.COLOR_RGB2BGR))
cv2.imwrite("l_interest.png", cv2.cvtColor(l_interest, cv2.COLOR_RGB2BGR))


ax[2, 0].imshow(r_64)
ax[2, 1].imshow(c_64)
ax[2, 2].imshow(l_64)
# tigth layout
plt.tight_layout()
plt.show()


# plt.imshow(np.concatenate((image1, get_small_image_center(image2), image3), axis=1))

In [ ]:
image1 = camera1.get_env_image()
image2 = camera2.get_env_image()
image3 = camera3.get_env_image()

r_64, r_interest = cut_image1(image1)
c_64, c_interest = cut_image2(image2)
l_64, l_interest = cut_image3(image3)


# load the interest areas (old images)
r_interest_old = cv2.imread("r_interest.png")
c_interest_old = cv2.imread("c_interest.png")
l_interest_old = cv2.imread("l_interest.png")
r_interest_old = cv2.cvtColor(r_interest_old, cv2.COLOR_BGR2RGB)
c_interest_old = cv2.cvtColor(c_interest_old, cv2.COLOR_BGR2RGB)
l_interest_old = cv2.cvtColor(l_interest_old, cv2.COLOR_BGR2RGB)
# r_interest_old = cv2.cvtColor(r_interest_old, cv2.COLOR_BGR2RGB)
# Make sure r_interest, c_interest, and l_interest are defined elsewhere
# and contain the images you want to compare against the old ones.

# # comare the interest areas
# print(type(r_interest_old))
# print(r_interest_old.dtype)
# print(type(r_interest))
# print(r_interest.dtype)
diff_r = np.abs(r_interest_old / 255.0 - r_interest / 255.0) * 255
diff_c = np.abs(c_interest_old / 255.0 - c_interest / 255.0) * 255
diff_l = np.abs(l_interest_old / 255.0 - l_interest / 255.0) * 255

print("max diff r: ", diff_r.max())
print("max diff pixel r: ", diff_r.argmax())
print("max diff c: ", diff_c.max())
print("max diff l: ", diff_l.max())

print("average diff r: ", diff_r.mean())
print("average diff c: ", diff_c.mean())
print("average diff l: ", diff_l.mean())

# convert to uint8 for display
diff_r = diff_r.astype(np.uint8)
diff_c = diff_c.astype(np.uint8)
diff_l = diff_l.astype(np.uint8)

# clamp at 255 if necessary
diff_r = np.clip(diff_r, 0, 255)
diff_c = np.clip(diff_c, 0, 255)
diff_l = np.clip(diff_l, 0, 255)

fig, ax = plt.subplots(3, 3)
ax[0, 0].imshow(r_interest_old)  # Convert BGR to RGB for display
ax[0, 1].imshow(c_interest_old)
ax[0, 2].imshow(l_interest_old)
ax[1, 0].imshow(r_interest)
ax[1, 1].imshow(c_interest)
ax[1, 2].imshow(l_interest)
ax[2, 0].imshow(diff_r)
ax[2, 1].imshow(diff_c)
ax[2, 2].imshow(diff_l)
# average
# ax[3, 0].imshow(np.mean([r_interest_old, r_interest], axis=0).astype(np.uint8))
# ax[3, 1].imshow(np.mean([c_interest_old, c_interest], axis=0).astype(np.uint8))
# ax[3, 2].imshow(np.mean([l_interest_old, l_interest], axis=0).astype(np.uint8))
plt.show()

make sure you have to enable x11 forwarding
```bash
xhost +
```

In [12]:
import numpy as np
import time
import cv2
import time


def resize_and_pad(image, max_width, max_height, color=(0, 0, 0)):
    """
    Resizes and pads an image to fit within max_width and max_height while maintaining aspect ratio.

    Parameters:
        image (numpy.ndarray): The input image.
        max_width (int): The maximum width.
        max_height (int): The maximum height.
        color (tuple): The padding color in BGR. Default is black.

    Returns:
        numpy.ndarray: The resized and padded image.
    """
    original_height, original_width = image.shape[:2]

    # Calculate scaling factor to fit the image within max dimensions
    scale = min(max_width / original_width, max_height / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # Resize the image
    resized_image = cv2.resize(
        image, (new_width, new_height), interpolation=cv2.INTER_AREA
    )

    # Calculate padding to center the image
    delta_w = max_width - new_width
    delta_h = max_height - new_height
    top = delta_h // 2
    bottom = delta_h - top
    left = delta_w // 2
    right = delta_w - left

    # Apply padding
    padded_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color
    )
    return padded_image


for i in range(1000):
    image1 = camera1.get_env_image()
    image2 = camera2.get_env_image()
    image3 = camera3.get_env_image()

    img_raw = np.concatenate((image1, image2, image3), axis=1)
    r_64, r_interest = cut_image1(image1)
    c_64, c_interest = cut_image2(image2)
    l_64, l_interest = cut_image3(image3)

    r_interest = cv2.resize(r_interest, (64 * 2, 64 * 2), interpolation=cv2.INTER_AREA)
    c_interest = cv2.resize(c_interest, (64 * 2, 64 * 2), interpolation=cv2.INTER_AREA)
    l_interest = cv2.resize(l_interest, (64 * 2, 64 * 2), interpolation=cv2.INTER_AREA)

    img_interest = np.concatenate((r_interest, c_interest, l_interest), axis=1)
    img_64 = np.concatenate((r_64, c_64, l_64), axis=1)

    img_interest = resize_and_pad(img_interest, img_raw.shape[1], img_raw.shape[0])
    img_64 = resize_and_pad(img_64, img_raw.shape[1], img_raw.shape[0])

    img_all = np.concatenate((img_raw, img_interest, img_64), axis=0)

    cv2.imshow("images", img_all)
    cv2.waitKey(1)
    time.sleep(0.1)
cv2.destroyAllWindows()

In [ ]:
from FlexivPy.controllers.jointspace import GoJointConfigurationVelocity
from FlexivPy.controllers.runners import blocking_runner

state = robot.get_robot_state()
info = model.getInfo(np.array(state.q), np.array(state.dq))
print(info["poses"]["link7"])
print(state.q)


# homing_controller = GoJointConfigurationVelocity(qgoal=np.array([-0.060038141906261444, -0.1400885432958603, 0.11492829769849777, 2.026796817779541, 0.0031041193287819624, 0.5770605802536011, 0.]))
# blocking_runner(robot, homing_controller)

# default T is :

target_t = np.array(
    [
        [1, 0, 0, 0.48986543],
        [0, -1, 0, -0.08922464],
        [0, 0, -1, 0.65726634],
        [0.0, 0.0, 0.0, 1.0],
    ]
)


q0 = np.array(
    [
        -0.006483817007392645,
        -0.2647894322872162,
        0.1409081667661667,
        1.9820226430892944,
        -0.04699726402759552,
        0.6730022430419922,
        0.1588847041130066,
    ]
)

pre_q0 = np.array(
    [
        -0.060038141906261444,
        -0.1400885432958603,
        0.11492829769849777,
        2.026796817779541,
        0.0031041193287819624,
        0.5770605802536011,
        0.0,
    ]
)

blocking_runner(robot, GoJointConfigurationVelocity(qgoal=pre_q0, max_v=0.05))
blocking_runner(robot, GoJointConfigurationVelocity(qgoal=q0, max_v=0.05))

state = robot.get_robot_state()
info = model.getInfo(np.array(state.q), np.array(state.dq))

T0_ref = np.array(
    [
        [1.00000000e00, -2.69497544e-06, 2.74733386e-05, 5.31326556e-01],
        [-2.69558521e-06, -1.00000000e00, 2.21951225e-05, -4.72725715e-02],
        [2.74732787e-05, -2.21951966e-05, -9.99999999e-01, 6.27520138e-01],
        [0.00000000e00, 0.00000000e00, 0.00000000e00, 1.00000000e00],
    ]
)

if np.linalg.norm(T0_ref - info["poses"]["link7"]) > 1e-2:
    print(
        "error in the robot position", np.linalg.norm(T0_ref - info["poses"]["link7"])
    )
    print("The robot is not at the correct position")

In [ ]:
from FlexivPy.controllers.taskspace import (
    DiffIKController,
    Get_T_from_controller_no_drift,
)
import imageio
import pickle
import numpy as np
from PIL import Image
import pathlib

from datetime import datetime
import json
from concurrent.futures import ThreadPoolExecutor
from zoneinfo import ZoneInfo
from FlexivPy.controllers.runners import blocking_runner


# state = robot.get_robot_state()
# info = model.getInfo(np.array(state.q), np.array(state.dq))
# T0_ref = info["poses"]["link7"]


get_T_from_controller_no_drift = Get_T_from_controller_no_drift(
    T0=T0_ref, joy=joy, max_v=0.35
)

task_controller = DiffIKController(
    model,
    T_cmd_fun=get_T_from_controller_no_drift,
    dt=0.01,
    dq_max=1.0,
    control_mode="velocity",
)


def save_image_pil(im, fout):
    # A helper function to save the image
    im.save(fout)


class CallbackFun:
    def __init__(self, base_folder_name, max_workers_pool=1):
        self.cmds = []
        self.callback_dt = 0.1
        self.states = []
        self.img_names = []
        self.raw_img_names = []
        self.last_callback_t = -1
        self.img_counter = 0

        self.img_dir = base_folder_name / "img/"
        self.img_raw_dir = base_folder_name / "img_raw/"
        self.data_dir = base_folder_name / "data/"

        (self.img_dir / "cam1").mkdir(parents=True, exist_ok=True)
        (self.img_dir / "cam2").mkdir(parents=True, exist_ok=True)
        (self.img_dir / "cam3").mkdir(parents=True, exist_ok=True)

        self.data_dir.mkdir(parents=True, exist_ok=True)

        (self.img_raw_dir / "cam1").mkdir(parents=True, exist_ok=True)
        (self.img_raw_dir / "cam2").mkdir(parents=True, exist_ok=True)
        (self.img_raw_dir / "cam3").mkdir(parents=True, exist_ok=True)

        # Create a thread pool executor for asynchronous saving
        self.executor = ThreadPoolExecutor(max_workers=max_workers_pool)

    def save(self):
        """ """

        Dataout = {
            "q": [[i for i in x.q] for x in callback.states],
            "dq": [[i for i in x.dq] for x in callback.states],
            "imgs": callback.img_names,
            "raw_imgs": callback.raw_img_names,
        }

        with open(self.data_dir / "data.pkl", "wb") as f:
            pickle.dump(Dataout, f)

        with open(self.data_dir / "data.json", "w") as f:
            json.dump(Dataout, f)

        # generate a video in mp4 format.
        for cam_name in ["cam1", "cam2", "cam3"]:
            images = []
            for img in self.img_names:
                images.append(imageio.imread(img[cam_name]))
            imageio.mimsave(
                self.img_dir / cam_name / f"vid_{cam_name}.mp4",
                images,
                fps=1.0 / self.callback_dt,
                codec="h264",
            )
            images_raw = []
            for img in self.raw_img_names:
                images_raw.append(imageio.imread(img[cam_name]))
            imageio.mimsave(
                self.img_raw_dir / cam_name / f"vid_raw_{cam_name}.mp4",
                images_raw,
                fps=1.0 / self.callback_dt,
                codec="h264",
            )

    def __call__(self, robot, cmd, _):
        tic = time.time()
        if tic - self.last_callback_t > self.callback_dt:
            self.last_callback_t = tic

            fcam1 = self.img_dir / "cam1" / f"cam1_{self.img_counter:09d}.png"
            fcam2 = self.img_dir / "cam2" / f"cam2_{self.img_counter:09d}.png"
            fcam3 = self.img_dir / "cam3" / f"cam3_{self.img_counter:09d}.png"

            fcam1_big = (
                self.img_raw_dir / "cam1" / f"cam1_raw_{self.img_counter:09d}.jpg"
            )
            fcam2_big = (
                self.img_raw_dir / "cam2" / f"cam2_raw_{self.img_counter:09d}.jpg"
            )
            fcam3_big = (
                self.img_raw_dir / "cam3" / f"cam3_raw_{self.img_counter:09d}.jpg"
            )

            self.img_names.append(
                {"cam1": str(fcam1), "cam2": str(fcam2), "cam3": str(fcam3)}
            )
            self.raw_img_names.append(
                {
                    "cam1": str(fcam1_big),
                    "cam2": str(fcam2_big),
                    "cam3": str(fcam3_big),
                }
            )

            self.cmds.append(cmd)
            self.states.append(robot.get_robot_state())

            # store the image as png.
            img1 = camera1.get_env_image()
            img2 = camera2.get_env_image()
            img3 = camera3.get_env_image()

            img1_64, _ = cut_image1(img1)
            img2_64, _ = cut_image2(img2)
            img3_64, _ = cut_image3(img3)

            self.executor.submit(save_image_pil, Image.fromarray(img1_64), fcam1)
            self.executor.submit(save_image_pil, Image.fromarray(img2_64), fcam2)
            self.executor.submit(save_image_pil, Image.fromarray(img3_64), fcam3)

            self.executor.submit(
                save_image_pil,
                Image.fromarray(img1),
                fcam1_big,
            )
            self.executor.submit(
                save_image_pil,
                Image.fromarray(img2),
                fcam2_big,
            )
            self.executor.submit(
                save_image_pil,
                Image.fromarray(img3),
                fcam3_big,
            )

            if self.executor._work_queue.qsize() > 100:
                print(
                    "pool to save img to disk queue is big",
                    self.executor._work_queue.qsize(),
                )

            self.img_counter += 1


# Get local time zone (replace with your desired time zone if needed)
local_timezone = ZoneInfo("America/New_York")  # Replace with your timezone

# Get current date and time
current_datetime = datetime.now(local_timezone)

# Format the datetime string for folder name
folder_name = current_datetime.strftime("%Y-%m-%d__%H-%M-%S")
base_folder_name = pathlib.Path("data_t_v1") / folder_name

callback = CallbackFun(base_folder_name=base_folder_name, max_workers_pool=1)

status = blocking_runner(robot, task_controller, timeout=60, callback=callback)

In [ ]:
# save the data to disk (image names and robot states)
callback.save()